In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization,Dense,Conv2D,Reshape,Conv2DTranspose,ReLU,LeakyReLU,Flatten,Activation,Dropout
from tensorflow.keras.optimizers import  Adam
from tensorflow.keras.losses import BinaryCrossentropy

In [10]:
class DCGAN:
  def __init__(self,training_images):

    ### Considering channels last

    self.no_of_samples=training_images.shape[0]
    self.height=training_images.shape[1]
    self.width=training_images.shape[2]
    self.channels=training_images.shape[3]
    self.train_data=(training_images-127.5)/127.5 ### Converting grey scale [0-255] -> [-1,1]
    self.shape=(self.height,self.width,self.channels)
    self.noise_size=100
    self.Discriminator=None
    self.Generator=None
    self.disc_optimizer=Adam(learning_rate=0.0002,beta_1=0.5)
    self.gen_optimizer=Adam(learning_rate=0.0002,beta_1=0.5)
    self.loss=BinaryCrossentropy(from_logits=True)



  def get_generator(self):

    ### Generator definition

    w_init=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

    Generator= Sequential()

    ### Input layer: takes in input a random noise of 100 points distributed in a random distribution 
    Generator.add(Dense(int(self.height/4)*int(self.width/4)*256,use_bias=False, input_shape=(self.noise_size,),kernel_initializer=w_init))
    Generator.add(BatchNormalization())
    Generator.add(ReLU())

    ### Reshaping layer to reshape in image dimension
    Generator.add(Reshape((int(self.height/4),int(self.width/4),256)))

    ### Upconv layer 1  ## The size remains constant (7 x 7 x 128)
    Generator.add(Conv2DTranspose(128, (5,5), strides=(1,1),padding="same",kernel_initializer=w_init,use_bias=False))
    Generator.add(BatchNormalization())
    Generator.add(ReLU())

    ### Upconv layer 2  ## The size upsamples by 2 (14 x 14 x 128)
    Generator.add(Conv2DTranspose(64, (5,5), strides=(2,2),padding="same",use_bias=False,kernel_initializer=w_init))
    Generator.add(BatchNormalization())
    Generator.add(ReLU())

    ### Upconv layer 3  ## The size upsamples by 2 (28 x 28 x 1)
    Generator.add(Conv2DTranspose(self.channels, (5,5), strides=(2,2),padding="same",kernel_initializer=w_init,activation="tanh"))

    ### generator output must be of the dimension of the input shape of the image data: (28,28,1)
    ### The last layer uses tanh activation in the generator. 
    return Generator

  def get_discriminator(self):    
    
    ### Discriminator definition
    w_init=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

    Discriminator= Sequential()

    ### Input layer: Conv layer 1
    Discriminator.add(Conv2D(64, (5,5), strides=(2,2),input_shape=self.shape,padding="same",use_bias=False,kernel_initializer=w_init))
    Discriminator.add(BatchNormalization())
    Discriminator.add(LeakyReLU(0.2))

    ### Conv layer 2
    Discriminator.add(Conv2D(128, (5,5), strides=(2,2),padding="same",use_bias=False,kernel_initializer=w_init))
    Discriminator.add(BatchNormalization())
    Discriminator.add(LeakyReLU(0.2))

    ### Flatten
    Discriminator.add(Flatten())

    ### Prediction layer
    Discriminator.add(Dense(1,kernel_initializer=w_init))

    # Discriminator compilation

    return Discriminator
  def get_all_models(self):

    self.Discriminator=self.get_discriminator()

    self.Generator=self.get_generator()

    ### Compiling discriminator

    print("------------ DISCRIMINATOR------------------")
    print(self.Discriminator.summary())
    print("------------ GENERATOR------------------")
    print(self.Generator.summary())
    
    return None
  def train_on_batch_disc(self,data,labels):
    
    with tf.GradientTape() as disc_tape:
      output=self.Discriminator(data)
      loss_disc=self.loss(labels,output)
    
    d_grads=disc_tape.gradient(loss_disc,self.Discriminator.trainable_variables)
    self.disc_optimizer.apply_gradients(zip(d_grads,self.Discriminator.trainable_variables))

    pred_prob=tf.round(tf.nn.sigmoid(self.Discriminator(data,training=False)))
    disc_acc=tf.reduce_mean(tf.cast(tf.equal(labels,pred_prob),dtype=tf.float32))
    acc=disc_acc.numpy()
    return loss_disc,acc
  
  def train_on_batch_gen(self,data,labels):
    
    with tf.GradientTape() as gen_tape:

      output=self.Discriminator(self.Generator(data,training=False))
      loss_gen=self.loss(labels,output)
    
    g_grads=gen_tape.gradient(loss_gen,self.Generator.trainable_variables)
    self.gen_optimizer.apply_gradients(zip(g_grads,self.Generator.trainable_variables))

    pred_prob=tf.round(tf.nn.sigmoid(self.Discriminator(self.Generator(data,training=False),training=False)))
    gen_acc=tf.reduce_mean(tf.cast(tf.equal(labels,pred_prob),dtype=tf.float32))
    acc=gen_acc.numpy()
    return loss_gen,acc

  def show_images(self, rows=4, columns=4):

    z = tf.random.uniform([rows*columns,self.noise_size])

    generated_images = self.Generator.predict(z)

    generated_images= (generated_images - (-1))/(1 - (-1))

    ### Min-Max scaling to convert pixles from [-1,1] -> [0,1]

    ### Plotting
    fig, ax = plt.subplots(rows,columns, figsize=(6,6))

    for i in range(rows):
      for j in range(columns):
        ax[i,j].imshow(generated_images[i+j,:,:,:])
    plt.show()
  
  def train(self,epochs,batch_size):

    self.get_all_models()  ### Initializing all models

    dataset=tf.data.Dataset.from_tensor_slices(self.train_data).shuffle(self.no_of_samples).batch(batch_size,
                                                                                                 drop_remainder=True)

    gen_loss=[]
    dis_loss=[]

    dis_accuracy=[]
    gen_accuracy=[]
    
    history={}

    no_of_batches=self.no_of_samples/batch_size

    real_labels=np.ones((batch_size,1)) ### For the real images, the label: 1
    ### The discriminator must predict 1 for the real images

    fake_labels=np.zeros((batch_size,1))### For the fake images, generated by the generator, the label: 0
    ### The discriminator must predict 0 for the fake images

    for epoch in range(epochs):      ### Training epochs
      d_loss=0
      g_loss=0
      d_acc=0
      g_acc=0

      for samples in dataset:

        ########    DISCRIMINATOR TRAINING    #########


        z = tf.random.uniform([batch_size,self.noise_size]) ### Produces batch_size number of 100 dimensional noise arrays of uniform distribution
        
        generated=self.Generator.predict(z)  ### obtaining the generated images

        training_set_of_discriminator=np.concatenate((samples,generated))
        labels_of_discriminator=np.concatenate((real_labels,fake_labels))

        loss_disc,acc_disc=self.train_on_batch_disc(training_set_of_discriminator,labels_of_discriminator)


        ########    GENERATOR TRAINING    #########


        loss_generator,acc_generator=self.train_on_batch_gen(z,real_labels) ### Discriminator is fixed, so the loss is actually genertors
      
        d_loss+=loss_disc
        g_loss+=loss_generator
        d_acc+=acc_disc
        g_acc+=acc_generator
      
      d_loss/=no_of_batches
      g_loss/=no_of_batches
      d_acc/=no_of_batches
      g_acc/=no_of_batches


      dis_loss.append(d_loss)
      dis_accuracy.append(d_acc)

      gen_loss.append(g_loss)
      gen_accuracy.append(g_acc)

      if epoch%20==0:
        #self.show_images()

        print(f"ON EPOCH {epoch} Discriminator Loss: {d_loss}, Discriminator accuracy: {d_acc}, Generator loss: {g_loss}, GAN accuracy: {g_acc}")
  
    history["Discriminator loss"]=dis_loss
    history["Discriminator accuracy"]=dis_accuracy

    history["Generator loss"]=gen_loss
    history["Generator accuracy"]=acc_generator
    return history

  

  
  

In [11]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.cifar10.load_data()



In [12]:
GAN=DCGAN(x_train)

In [ ]:
history=GAN.train(1000,128)